In [158]:
!wget https://github.com/flo7716/School-codes/blob/main/DL_B3/fleurs_mal.txt

--2025-04-07 21:17:51--  https://github.com/flo7716/School-codes/blob/main/DL_B3/fleurs_mal.txt
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘fleurs_mal.txt.2’

fleurs_mal.txt.2        [   <=>              ] 779.97K  1.36MB/s    in 0.6s    

2025-04-07 21:17:52 (1.36 MB/s) - ‘fleurs_mal.txt.2’ saved [798693]




<a id='chap-tpdeeplearning5'></a>

In [159]:
import tensorflow as tf
print(tf.test.is_built_with_cuda())  # Should return True
print(tf.config.list_physical_devices('GPU'))

True
[]


# Travaux pratiques - RNN pour la génération de texte

L’objectif de cette séance de travaux pratiques est d’illustrer la mise
en application des réseaux de neurones récurrents sur des données
séquentielles. En particulier, nous allons nous intéresser aux modèles
auto-régressifs pour la génération de texte.

## Génération de poésie

Une première application des réseaux de neurones récurents est la
génération de texte. Pour démarrer, nous allons extraire les textes d’un
recueil de poésies, « Les fleurs du mal » (1857) de l’écrivain [Charles
Baudelaire](https://fr.wikipedia.org/wiki/Charles_Baudelaire)
(1821-1867). Cet ensemble de textes va constituer notre corpus
d’entraînement.

Dans notre application, nous nous intéressons à la génération de texte
au travers de la *prédiction du mot suivant*. En considérant un texte
comme une suite de mots $ (x_1, x_2, ..., x_m) $, nous allons
entraîner un réseau de neurones récurrent de sorte à prédire le bon mot
$ x_n $ à partir des mots $ (x_1, x_2, ..., x_{n-1}) $ qui le
précèdent dans une phrase.

### Création du jeu de données d’entraînement

Le code ci-dessous va nous servir à générer les données et les
étiquettes correspondantes. On va commencer par parser le ficher
d’entrée pour récupérer le texte et effectuer quelques pré-traitements
simples:

In [160]:
# Lire le fichier texte et ajouter toutes les lignes dans une liste
with open("fleurs_mal.txt", 'r' , encoding = 'utf8') as f:
    lines = f.readlines()

for idx, line in enumerate(lines):
    if "Charles Baudelaire avait un ami" in line:
        first_line = idx
    if "End of the Project Gutenberg EBook of Les Fleurs du Mal, by Charles Baudelaire" in line:
        last_line = idx

lines = lines[first_line:last_line]
lines = [l.lower().strip().replace('_', '') for l in lines if len(l) > 1]
text = " ".join(lines)

In [161]:
characters = sorted(set(text))
n_characters = len(characters)

In [162]:
text[:258]

"charles baudelaire avait un ami, auguste poulet-malassis, ancien élève de l'école des chartes, qui s'était fait éditeur par goût pour les raffinements typographiques et pour la littérature qu'il jugeait en érudit et en artiste beaucoup plus qu'en commerçant;"

### Question

Que contient la variable `characters` ? Que représente
`n_characters` ? La documentation des
[Set](https://docs.python.org/fr/3/tutorial/datastructures.html#sets)
en Python peut vous aider.

Dans la suite de ce TP, nous allons considérer le texte comme une suite
de caractères. Nous n’allons donc pas raisonner au niveau du mot mais au
niveau du symbole. Chaque caractère du texte d’entrée sera représenté en
entrée du réseau de neurones par un encodage *one-hot* sur le
dictionnaire de symboles. Autrement dit, pour un dictionnaire simplifié
(” “, `a`, `b`, `c`, `d`), la lettre `a` serait représentée
par le vecteur $ (0, 1, 0, 0, 0, 0) $ tandis que l’espace “ “
serait représenté par le vecteur $ (1, 0, 0, 0, 0) $.

Nous allons désormais entraîner un réseau de neurones récurrent.
Celui-ci va recevoir en entrée une séquence de `SEQLEN` caractères.
Son objectif sera de prédire en sortie le caractère suivant dans le
corpus. Par exemple, pour la phrase :

> Le vélo est rouge.


le modèle devra prédire `l` à partir de la séquence `Le vé`, puis
`o` à partir de la séquence `Le vél`, et ainsi de suite. Il s’agit
donc d’un problème de classification à `n_characters` classes
différentes (une classe par symbole).

L’étiquette de classe est obtenue automatiquement à partir du corpus.
Comme il n’y a eu aucune annotation manuelle du jeu de données, cet
objectif de prédiction du caractère suivant représente un problème
d’apprentissage dit *auto-supervisé* (ou *self-supervised*). La
supervision est construite artificiellement à partir des données elles
mêmes.

Les données d’entraînement consistent donc en l’ensemble des séquences
de caractères du corpus dont la taille est inférieure à `SEQLEN`.
L’étiquette de la classe cible correspondante est celle de l’indice du
prochain caractère à prédire, c’est-à-dire le caractère suivant dans le
corpus.

In [163]:
# SEQLEN représente la taille de la séquence de lettres à passer en entrée
SEQLEN = 10
step = 1
input_characters, labels = [], []
# On parcourt le corpus de texte avec une fenêtre glissante
for i in range(0, len(text) - SEQLEN, step):
    input_characters.append(text[i:i + SEQLEN])
    labels.append(text[i + SEQLEN])

print(f"Il y a {len(input_characters)} séquences de {SEQLEN} caractères dans le corpus d'entraînement.")

Il y a 146120 séquences de 10 caractères dans le corpus d'entraînement.


In [164]:
input_characters[0], labels[0]

('charles ba', 'u')

### Question

Afficher une séquence de `SEQLEN` caractères et l’étiquette de classe
correspondante, c’est-à-dire le caractère suivant.

In [165]:
print(f"Il y a {n_characters} caractères uniques dans le corpus d'entraînement.")

Il y a 59 caractères uniques dans le corpus d'entraînement.


Nous pouvons maintenant vectoriser les données d’entraînement en
utilisant le dictionnaire et un encodage *one-hot* pour chaque caractère
:

In [166]:
# Encodage caractère -> indice du dictionaire
char2index = dict((c, i) for i, c in enumerate(characters))
# Encodage de l'indice vers le caractère (utilisé pour décoder les prédictions du modèle)
index2char = dict((i, c) for i, c in enumerate(characters)) # mapping index -> char in dictionary

Chaque séquence d’entraînement est donc représentée par une matrice de
taille $ SEQLEN \times m $, correspondant à une longueur de
`SEQLEN` caractères, chaque caractère étant encodé par un vecteur
binaire correspondant à un encodage *one-hot*. $ m $ représente la
taille du dictionnaire, c’est-à-dire le nombre de symboles uniques dans
le corpus.

- L’ensemble des données d’entraînement `X` seront donc constituées
  par un tenseur de taille $ N \times SEQLEN \times m $ où $ N $
  est le nombre de séquences de `SEQLEN` caractères dans le corpus.  
- L’ensemble des labels d’entraînement `y` seront représentées par un
  tenseur de $ N \times m $, où la sortie pour chaque
  exemple correspond à l’indice dans le dictionnaire du caractère
  suivant la séquence  

### Question

Compléter le code ci-dessous afin de créer les tenseurs `X` et `y`
contenant les données d’entraînement (séquences de caractères dans `X`
et étiquettes de classe dans `y`. Vous pourrez notamment utiliser à
bon escient le dictionnaire `char2index` qui permet de transformer un
caractère en son indice entier.

In [167]:
import numpy as np
X = np.zeros((len(input_characters), SEQLEN, n_characters), dtype=bool)
y = np.zeros((len(input_characters), n_characters), dtype=bool)

for idx_seq, sequence in enumerate(input_characters):
    # À compléter
    # Remplissage des tenseurs X et y
    for idx_char, char in enumerate(sequence):
        # Encodage one-hot dans X
        X[idx_seq, idx_char, char2index[char]] = 1

    next_char = labels[idx_seq]  # Récupérer le caractère cible
    # Encodage one-hot dans y
    y[idx_seq, char2index[next_char]] = 1

Comme à l’accoutumée, nous allons séparer le jeu de données en deux : un
ensemble d’apprentissage et un ensemble de validation. Le jeu de
validation nous permettra notamment d’évaluer les performances du modèle
et d’éviter le sur-apprentissage.

In [168]:
from sklearn.model_selection import train_test_split

# 90% des données en apprentissage, 10% en validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
y_train.shape

(131508, 59)

### Apprentissage d’un modèle auto-supervisé pour la génération de texte

Maintenant que les données ont été formatées, nous pouvons commencer à
définir le modèle que nous allons utiliser. Nous allons l’implémenter
sous la forme d’un modèle Keras séquentiel
([Sequential](https://keras.io/api/models/sequential/)).

In [169]:
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.models import Sequential

model = Sequential()


Pour l’instant, ce modèle est vide. Nous allons lui ajouter une couche
récurrente avec un modèle de type `SimpleRNN` (la cellule récurrente
la plus simple) :

In [170]:
h_size = 59
model.add(SimpleRNN(h_size, return_sequences=False, unroll=True))

### Question

À l’aide de la documentation de
[SimpleRNN](https://keras.io/api/layers/recurrent_layers/simple_rnn/)
dans Keras, expliquer à quoi correspondent les paramètres
`h_size = 128` et `return_sequences=False`.

### Note

L’argument optionnel `unroll=True` permet simplement
d’accélérer les calculs en « déroulant » le réseau récurrent plutôt que
d’utiliser une boucle `for` en interne.

Pour terminer notre modèle, nous ajoutons enfin une couche entièrement
connectée suivie d’une fonction `softmax` qui effectuera la
classification du caractère suivant la séquence.

In [171]:
from tensorflow.keras.layers import Dense, Activation

model.add(Dense(h_size))
model.add(Activation("softmax"))

Empiriquement, il a été constaté que l’optimisation des réseaux
récurrents est plus rapide et la convergence plus robuste lorsque l’on
utilise des méthodes de descente de gradient à pas adaptatif, telles que
`RMSprop` [[TH12]](#tieleman2012).

In [172]:
from tensorflow.keras.optimizers import RMSprop

learning_rate = 0.001
optim = RMSprop(learning_rate=learning_rate)

Nous pouvons donc compiler le modèle et utiliser la méthode
`summary()` de Keras pour visualiser le nombre de paramètres du
réseaux

In [173]:
# Build the model by specifying the input shape
model.build(input_shape=(None, SEQLEN, n_characters))
# Compile the model
model.compile(loss="categorical_crossentropy", optimizer=optim, metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_11 (SimpleRNN)   (None, 59)                7021      
                                                                 
 dense_9 (Dense)             (None, 59)                3540      
                                                                 
 activation_9 (Activation)   (None, 59)                0         
                                                                 
Total params: 10561 (41.25 KB)
Trainable params: 10561 (41.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


L’entraînement s’effectue de la manière habituelle à l’aide de la
méthode `fit()`:

In [174]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((131508, 10, 59), (131508, 59), (14612, 10, 59), (14612, 59))

In [175]:
batch_size = 128
num_epochs = 50
model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs)

Epoch 1/50
1028/1028 [==============================] - 3s 3ms/step - loss: 2.5302 - accuracy: 0.2805
Epoch 2/50
1028/1028 [==============================] - 3s 3ms/step - loss: 2.2086 - accuracy: 0.3415
Epoch 3/50
1028/1028 [==============================] - 3s 3ms/step - loss: 2.1313 - accuracy: 0.3601
Epoch 4/50
1028/1028 [==============================] - 3s 3ms/step - loss: 2.0828 - accuracy: 0.3719
Epoch 5/50
1028/1028 [==============================] - 3s 3ms/step - loss: 2.0448 - accuracy: 0.3816
Epoch 6/50
1028/1028 [==============================] - 3s 3ms/step - loss: 2.0140 - accuracy: 0.3904
Epoch 7/50
1028/1028 [==============================] - 3s 3ms/step - loss: 1.9871 - accuracy: 0.3970
Epoch 8/50
1028/1028 [==============================] - 3s 3ms/step - loss: 1.9647 - accuracy: 0.4034
Epoch 9/50
1028/1028 [==============================] - 3s 3ms/step - loss: 1.9447 - accuracy: 0.4093
Epoch 10/50
1028/1028 [==============================] - 3s 3ms/step - loss: 1.927

Nous pouvons utiliser la méthode `.evaluate()` pour calculer
automatiquement les métriques spécifiées à la compilation du modèle sur
un jeu de données. Calculons par exemple les scores de taux de bonne
classification (*accuracy*) sur le jeu d’apprentissage et sur le jeu de
test :

In [176]:
scores_train = model.evaluate(X_train, y_train, verbose=0)
scores_test = model.evaluate(X_test, y_test, verbose=0)
print(f"Performances (apprentissage, {model.metrics_names[1]}) = {scores_train[1]*100:.2f}")
print(f"Performances (validation, {model.metrics_names[1]}) = {scores_test[1]*100:.2f}")

Performances (apprentissage, accuracy) = 47.32
Performances (validation, accuracy) = 45.78


*Note*: Il est possible de sauvegarder les paramètres du modèle appris à
l’aide de la méthode `.save()`. Cela permet notamment de réutiliser le
modèle plus tard, sans avoir à l’entraîner de nouveau.

In [177]:
model_name = f"SimpleRNN_{h_size}_{num_epochs}epochs.keras"
model.save(model_name)

### Analyse de l’apprentissage

### Question

Quels taux de classification obtient-on en apprentissage ? En validation
?

### Question

En quoi le problème est-il différents des problèmes de classification
abordés jusqu’ici ? Par exemple, faire une recherche de la séquence
d’entrée « la mort de », et analyser les labels cibles présents dans le
corpus d’apprentissage.

### Génération de texte à partir du modèle appris

Nous pouvons désormais nous servir du modèle entraîné pour générer du
texte qui va « imiter » le style du corpus de poésie initial (*Les
Fleurs du mal*). Si nécessaire, commençons par charger les paramètres du
réseau récurrent précédemment entraîné à l’aide de la fonction
`loadModel` :

In [178]:
from tensorflow.keras.models import load_model
model = load_model(model_name)
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_11 (SimpleRNN)   (None, 59)                7021      
                                                                 
 dense_9 (Dense)             (None, 59)                3540      
                                                                 
 activation_9 (Activation)   (None, 59)                0         
                                                                 
Total params: 10561 (41.25 KB)
Trainable params: 10561 (41.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Comme le modèle a été entraîné pour prédire le caractère suivant à
partir d’une séquence de `SEQLEN` caractères précédent, nous devons
l’initialiser avec une chaîne de caractères de départ. Le modèle pourra
ensuite générer du texte en prédisant les caractères suivants un par un.

Reprenons un texte initial issu de notre corpus d’entraînement :

In [179]:
idx= 10
# index2char permet de repasser de l'encodage one-hot au caractère du dictionnaire
initial_characters = [index2char[np.argmax(c)] for c in X_train[idx]]
initial_text = "".join(initial_characters)
print(f"La séquence n°{idx} est : '{initial_text}'")

La séquence n°10 est : ' qu'une cl'


Nous pouvons maintenant extraire la représentation en encodage *one-hot*
de ce texte, que nous passerons ensuite dans le réseau entraîné pour
obtenir la prédiction du caractère suivant.

In [180]:
test_sequence = np.zeros((1, SEQLEN, n_characters), dtype=bool)
test_sequence[0] = X_train[idx]
prediction = model.predict(test_sequence)
print(prediction)

1/1 [==============================] - 0s 89ms/step
[[2.7085681e-04 1.3478141e-05 2.5185312e-03 3.2861404e-05 4.3679589e-05
  6.7043140e-05 1.4864804e-05 7.9784288e-05 1.3624537e-05 2.2148219e-05
  1.4267722e-05 2.2317694e-05 1.7991355e-05 3.0723637e-05 1.5316844e-05
  2.3664103e-05 1.5681208e-05 7.9908561e-05 4.1811535e-01 1.4498232e-04
  3.3417883e-04 2.9774907e-05 2.7154067e-01 1.5574973e-04 7.2341616e-05
  3.3703807e-04 3.7491906e-02 1.0971951e-04 1.9330128e-05 3.4377684e-03
  8.0702396e-04 2.1660653e-05 2.0787546e-01 2.0422128e-03 1.7956585e-04
  6.6160275e-05 3.7163907e-05 1.2088626e-04 9.2207938e-03 6.5577674e-06
  2.5785388e-05 1.1815487e-06 3.6195945e-04 4.8781089e-06 1.3710106e-05
  4.1847688e-06 3.6174909e-04 5.9232046e-03 7.0270221e-06 2.5081465e-03
  3.4678955e-02 3.0499100e-04 5.0131126e-05 2.2016009e-04 1.1638280e-05
  2.1730608e-05 2.4951557e-06 2.1087732e-05 1.3869984e-05]]


### Question

À l’aide du dictionnaire `index2char`, déterminer quel
est le prochain caractère prédit par le modèle.

Au lieu de prédire systématiquement le symbole dont la probabilité est
maximale, nous pouvons ajouter du non-déterministe (et donc de
l’aléatoire) dans la génération de texte en échantillonnant selon la
distribution de probabilités en sortie du softmax. Autrement dit, plus
un symbole aura une forte activation après le softmax, plus il aura de
chances d’être tiré.

Pour contrôler à quel point cet échantillonnage sera non-déterministe,
nous allons introduire un nouveau paramètre permettant de contrôler la
forme de la distribution. En notant $ T $ ce nouveau paramètre, nous
allons altérer les probabilités de sortie en les remplaçant par la
formule suivante :

$$
z_{i}^N  = \frac{z_{i}^{\frac{1}{T}}}{\sum\limits_{j=1}^C z_{j}^{\frac{1}{T}} }
$$

$ T $ est un paramètre appelé *température*. Si $ T=1 $, alors
il s’agit du softmax habituel.

La figure ci-dessous montre l’impact sur la distribution de cette
renormalisation :

Nous pourrons par la suite utiliser la fonction ci-dessous qui tire
aléatoirement un symbole en échantillonnant selon la distribution,
éventuellement modifiée par la température.

In [181]:
def sample(probabilities, temperature=1.0):
    probabilities = np.asarray(probabilities).astype('float64')
    # Modifie la distribution selon la valeur de la température
    probabilities = pow(probabilities, 1.0/temperature)
    probabilities /= np.sum(probabilities)
    # Tire des variables aléatoires selon la distribution multinomiale transformée
    random_values = np.random.multinomial(1, probabilities, 1)
    # Renvoie le symbole échantillonné
    return np.argmax(random_values)

### Question

Comment est-ce que la température modifie la distribution
lorsqu’elle augmente ($ T \rightarrow +\infty $) ou
diminue ($ T \rightarrow 0 $) ?
Comment cela va-t-il influer sur l’échantillonnage du caractère suivant ?

Pour terminer, nous pouvons mettre en place la génération de texte à
partir d’une séquence de `SEQLEN` caractères initiaux. Pour ce faire,
nous allons créer une boucle qui :

1. extraie les `SEQLEN` derniers caractères du texte généré  
1. calcule les probabilités après softmax du réseau (méthode
  `.predict()`)  
1. échantillonne un caractère dans ces probabilités (fonction
  `sample()`)  
1. ajoute ce caractère au texte généré  

### Question

Compléter le code de génération de texte ci-dessous.

In [182]:
# Longueur du texte à générer (en caractères)
text_length = 200
# Température
temperature  = 0.9


generated_text = initial_text
network_input = test_sequence

for i in range(text_length):
    last_characters = generated_text[-SEQLEN:]  # Séquence d'entrée
    # Compléter le code
    # Transformer la séquence en encodage one-hot
    x_pred = np.zeros((1, SEQLEN, n_characters), dtype=bool)
    for t, char in enumerate(last_characters):
        x_pred[0, t, char2index[char]] = 1

    # Prédire la distribution des probabilités des prochains caractères
    predictions = model.predict(x_pred, verbose=0)[0]

    # Échantillonnage du prochain caractère à l'aide de la fonction sample
    next_index = sample(predictions, temperature)
    next_char = index2char[next_index]

    # Ajout du caractère généré au texte
    generated_text += next_char

    # Mise à jour de l'entrée pour la prochaine itération
    network_input = list(network_input[1:]) + [next_char]


print("Texte généré :")
print(generated_text)


Texte généré :
 qu'une cleubles qui nousfurs incumbe: j'est les choondai tout tes vidin comme des ne fourde mon coeurs ces leinules que cette les ablemes crésus gréter ayprés pas comme un rigner et forteur et décîme au gormin


### Question

Évaluer l’impact du paramètre de température dans la génération,
ainsi que le nombre d’époques dans l’apprentissage.
Commenter les points forts et points faibles du générateur.

## Pour aller plus loin

Comment doit-on modifier le jeu de données ou modèle de RNN pour
améliorer les performances du générateur ? Essayez notamment d’augmenter
`SEQLEN`. Quel problème cela résout-il ? Un modèle avec plus de
paramètres donne-t-il de meilleurs résultats ?